<a href="https://colab.research.google.com/github/msamwelmollel/ML-CLUB-2024-Eagle-Labs/blob/main/ML_CLUB_2024_Eagle_Labs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -qU langchain langchain-openai langchain-mongodb langchain-experimental ragas pymongo tqdm